# Fine align
---

#### Overview
Interactive 3D alignment of serial sections.

In [1]:
import pathlib
import requests

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import renderapi
import os

from scripted_render_pipeline import basic_auth
from scripted_render_pipeline.importer import uploader
from interactive_render import plotting
from interactive_render.utils import clear_image_cache

In [2]:
clear_image_cache()

<Response [200]>

#### `render-ws` environment variables

In [3]:
# create an authorized session
auth = basic_auth.load_auth()
sesh = requests.Session()
sesh.auth = auth

# render-ws environment variables
params_render = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "session": sesh
}

params_uploader = {
    "host": "https://sonic.tnw.tudelft.nl",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "auth": auth
}

# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/")

## 5) Fine alignment
---

In [7]:
stacks = {
    'rough_align': 'postcorrection_aligned_rough',
    'in': 'postcorrection',
    'out': 'postcorrection_rigid'
}

### Get point matches

Use `render-ws` `PointMatchClient` script to find point matches between tilepairs in `z`, based on rough alignment
#### Collect tile pairs

In [17]:
# Get z values
z_values = renderapi.stack.get_z_values_for_stack(stack=stacks['rough_align'],
                                                  **params_render)

# Get tile pairs
tile_pairs= renderapi.client.tilePairClient(stack=stacks['rough_align'],
                                             minz=min(z_values),
                                             maxz=max(z_values),
                                             zNeighborDistance=1, # half-height of search cylinder
                                             xyNeighborFactor=0.1,
                                             excludeSameLayerNeighbors=True,
                                             subprocess_mode="check_output",  # suppresses output
                                             **params_render)
# Show tile pairs
out = f"Number of tile pairs... {len(tile_pairs['neighborPairs'])}"
print(out, "\n" + "-"*len(out))
tile_pairs['neighborPairs'][0:5]

Number of tile pairs... 70 
--------------------------


[{'p': {'groupId': 'S003', 'id': 't00_z0_y4_x4'},
  'q': {'groupId': 'S004', 'id': 't00_z1_y4_x4'}},
 {'p': {'groupId': 'S003', 'id': 't01_z0_y4_x3'},
  'q': {'groupId': 'S004', 'id': 't00_z1_y4_x4'}},
 {'p': {'groupId': 'S003', 'id': 't01_z0_y4_x3'},
  'q': {'groupId': 'S004', 'id': 't01_z1_y4_x3'}},
 {'p': {'groupId': 'S003', 'id': 't02_z0_y4_x2'},
  'q': {'groupId': 'S004', 'id': 't01_z1_y4_x3'}},
 {'p': {'groupId': 'S003', 'id': 't02_z0_y4_x2'},
  'q': {'groupId': 'S004', 'id': 't02_z1_y4_x2'}}]

In [31]:
# Name for pointmatch collection
match_collection = f"{params_render['project']}_{stacks['in']}_points"
match_collection

'20230914_RP_exocrine_partial_test_postcorrection_points'

#### Set `SIFT` & `RANSAC` parameters for fine alignment

In [32]:
from renderapi.client import (
    SiftPointMatchOptions,
    MatchDerivationParameters,
    FeatureExtractionParameters
)

In [35]:
# `RANSAC` parameters
params_RANSAC = MatchDerivationParameters(
    matchIterations=None,
    matchMaxEpsilon=25,        # maximal alignment error
    matchMaxNumInliers=None,
    matchMaxTrust=None,
    matchMinInlierRatio=0.01,  # minimal inlier ratio
    matchMinNumInliers=3,      # minimal number of inliers
    matchModelType='AFFINE',   # expected transformation
    matchRod=0.92              # closest/next closest ratio
)

# `SIFT` parameters
params_SIFT = FeatureExtractionParameters(
    SIFTfdSize=8,              # feature descriptor size
    SIFTmaxScale=0.2,         # (width/height *) maximum image size
    SIFTminScale=0.05,         # (width/height *) minimum image size
    SIFTsteps=7               # steps per scale octave
)

# Combined `SIFT` & `RANSAC` parameters
params_SIFT = SiftPointMatchOptions(
    fillWithNoise=True,
    **{**params_RANSAC.__dict__,
       **params_SIFT.__dict__}
)

# Extra parameters
params_SIFT.numberOfThreads = 1  # multithreading
params_SIFT.__dict__

{'SIFTfdSize': 8,
 'SIFTmaxScale': 0.2,
 'SIFTminScale': 0.05,
 'SIFTsteps': 7,
 'matchIterations': None,
 'matchMaxEpsilon': 25,
 'matchMaxNumInliers': None,
 'matchMaxTrust': None,
 'matchMinInlierRatio': 0.01,
 'matchMinNumInliers': 3,
 'matchModelType': 'AFFINE',
 'matchRod': 0.92,
 'renderScale': None,
 'fillWithNoise': True,
 'numberOfThreads': 1}

#### Get point matches

In [37]:
# Loop through tile pairs
for tp in tqdm(tile_pairs['neighborPairs']):

    # Format tile pair
    tp_ids = (tp["p"]["id"], tp["q"]["id"])

    # Run SIFT + RANSAC via render-ws PointMatchClient
    renderapi.client.pointMatchClient(
        stack=stacks['in'],
        collection=match_collection,
        tile_pairs=[tp_ids],
        sift_options=params_SIFT,
        excludeAllTransforms=True,
        subprocess_mode='check_output',  # suppresses output
        **params_render
    )

  0%|          | 0/70 [00:00<?, ?it/s]

### Align stack
---

### Create alignment files

In [ ]:
from pathlib import Path
import os
import subprocess
import json
from pprint import pprint

In [ ]:
z_values = renderapi.stack.get_z_values_for_stack(stacks['in'],
                                                  **params_render)
# Load align.json template
template_align_json = Path('../templates/align.json')
with template_align_json.open('r') as json_data:
    params_align_fine = json.load(json_data)

# Edit BigFeta solver schema
params_align_fine['first_section'] = min(z_values)
params_align_fine['last_section'] = max(z_values)
params_align_fine['solve_type'] = '3D'
params_align_fine['transformation'] = 'rigid'
params_align_fine['log_level'] = 'INFO'

# Edit input stack data
params_align_fine['input_stack']['host'] = params_render['host']
params_align_fine['input_stack']['owner'] = params_render['owner']
params_align_fine['input_stack']['project'] = params_render['project']
params_align_fine['input_stack']['name'] = stacks['in']

# Edit point match stack data
params_align_fine['pointmatch']['host'] = params_render['host']
params_align_fine['pointmatch']['owner'] = params_render['owner']
params_align_fine['pointmatch']['name'] = match_collection

# Edit output stack data
params_align_fine['output_stack']['host'] = params_render['host']
params_align_fine['output_stack']['owner'] = params_render['owner']
params_align_fine['output_stack']['project'] = params_render['project']
params_align_fine['output_stack']['name'] = stacks['out']

# Edit regularization parameters
params_align_fine['regularization']['default_lambda'] = 1e-2       # default: 0.005
params_align_fine['regularization']['translation_factor'] = 1e-2   # default: 0.005
params_align_fine['regularization']['thinplate_factor'] = 1e-5     # default: 1e-5

# Export montage settings to
align_json = dir_project / '_jsons_align_fine' / stacks['in'] / 'align_fine.json'
align_json.parent.mkdir(parents=True, exist_ok=True)
with align_json.open('w') as json_data:
    json.dump(params_align_fine, json_data, indent=2)

# Check alignment parameters
print(align_json)
print('-'*len(align_json.as_posix()))
pprint(params_align_fine)

### Run `BigFeta`

In [ ]:
# Path to `BigFeta`
cwd = Path.cwd().as_posix()
BigFeta_dir = Path('/home/catmaid/BigFeta/')

# Call `BigFeta.BigFeta` process -- have to switch to BigFeta directory
os.chdir(BigFeta_dir.as_posix())
subprocess.run(['python', '-m', 'bigfeta.bigfeta', '--input_json', align_json.as_posix()])
os.chdir(cwd)

### Inspect fine alignment

In [ ]:
# plot stack
plotting.plot_stacks(
    stacks=[stacks['out']],
    **params_render
)